# Quadrotor (aka "the drone")

## Set everything up

Import modules and configure the notebook.

In [1]:
import os

# This module is part of the python standard library
import time

# These modules are part of other existing libraries
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

# This is my own script (it is an interface to the pybullet simulator)
import ae353_drone

# I often go back and forth between making changes to my scripts and to
# the notebook in which they are used. One "gotcha" is that notebooks only
# import modules or scripts ONCE. Subsequent imports don't do anything, and
# in particular won't reflect any changes I've made to my scripts. To make
# sure that I'm working with the latest version of my code, I use this bit
# of magic, which forces the notebook to "reload" my script:
import importlib
importlib.reload(ae353_drone)

<module 'ae353_drone' from '/Users/timothybretl/Documents/courses/AE353/09 - AE353 (Spring 2021)/Website/examples/day40_drone/ae353_drone.py'>

Create simulator.

In [2]:
simulator = ae353_drone.Simulator(display=False)

## Create a controller

Define a controller.

In [3]:
class RobotController:
    def __init__(self, limiter=None):
        self.dt = 0.01
        self.limiter = limiter
        
        # Initialize record of old measurements
        self.r_old = 0.
        self.p_old = 0.
        self.y_old = 0.
        
        self.A = np.array([[0., 1.], [0., 0.]])
        self.B = np.array([[0.], [2.]])
        self.C = np.array([[1., 0.]])
        self.K = np.array([[9.999999999999996, 3.316624790355398]])
        self.L = np.array([[1.732050807568877], [0.9999999999999993]])
        
        self.f_z_e = 0.5 * 9.81
        self.z_e = 0.

    def get_color(self):
        return [0., 1., 0.]

    def reset(self, pos):
        self.xhat = np.zeros(2)

    def run(self, pos, rpy, pos_ring, is_last_ring, pos_others):
        
        ##################
        # PD Control
        #
        #  This is easy to implement and allows us to isolate the
        #  "z position and velocity" subsystem, for the purpose of
        #  example.
        #
        
        # Get current measurements of roll, pitch, and yaw
        r = rpy[0]
        p = rpy[1]
        y = rpy[2]
        
        # Estimate roll, pitch, and yaw derivatives by finite difference
        rdot = (r - self.r_old) / self.dt
        pdot = (p - self.p_old) / self.dt
        ydot = (y - self.y_old) / self.dt
        
        # Update record of old measurements
        self.r_old = r
        self.p_old = p
        self.y_old = y
        
        # Choose net torques to drive roll, pitch, and yaw to zero
        tau_x = - 1. * (r - 0) - 0.1 * (rdot - 0)
        tau_y = - 1. * (p - 0) - 0.1 * (pdot - 0)
        tau_z = - 1. * (y - 0) - 0.1 * (ydot - 0)
        
        #
        ##################
        
        zdes = pos_ring[2]
        zest = self.xhat[0] + self.z_e
        max_error = 0.25
        if np.abs(zdes - zest) > max_error:
            zdes = zest + max_error * ((zdes - zest) / linalg.norm(zdes - zest))
        
        
        xdes = np.array([zdes - self.z_e, 0.])
        u = -self.K @ (self.xhat - xdes)
        f_z = u[0] + self.f_z_e
        if self.limiter is not None:
            tau_x, tau_y, tau_z, f_z = self.limiter(tau_x, tau_y, tau_z, f_z)
        u[0] = f_z - self.f_z_e
        
        y = np.array([pos[2] - self.z_e])
        self.xhat += self.dt * (self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y))

        return tau_x, tau_y, tau_z, f_z

Define a variant of this controller as a subclass. This variant has a different $K$ matrix (just an example).

In [4]:
class MyOtherController(RobotController):
    def __init__(self, limiter=None):
        super().__init__(limiter)
        self.K = np.array([[5., 1.]])

Run the simulator with the first controller.

In [5]:
simulator.clear_drones()
simulator.add_drone(RobotController, 'my_netid', 'my_image.png')
simulator.reset()
simulator.run(max_time=5.0)

In [6]:
print(simulator.drones[0]['controller'].K)

[[10.          3.31662479]]


Run the simulator with the second controller.

In [7]:
simulator.clear_drones()
simulator.add_drone(MyOtherController, 'my_netid', 'my_image.png')
simulator.reset()
simulator.run(max_time=5.0)

In [8]:
print(simulator.drones[0]['controller'].K)

[[5. 1.]]
